# Running analysis on BugsInPy Dataset
## IMPORTANT: You need at least a T4 GPU
### Overall flow of the notebook:

1. Clone the project
2. Install dependencies
3. Chunk the error traces and make an index
4. Search the index for errotraces against code chunks
5. Generate statistics based on the results


### Importing all of the necessary modules

In [2]:
from scripts.embedding import model, MODEL_NAME, BATCH_SIZE, embed, index_embeddings
from scripts.bugsinpy_utils import *

import faiss
import numpy as np
import json, os

/home/pooria/bootcamp/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Making sure the directories exist

In [3]:
os.makedirs(os.path.abspath(f"tmp/ast/results"), exist_ok=True)
K = 60

### MRR & MPR

In [ ]:
def compute_average_precision(y_true):
    hits = 0
    sum_precisions = 0.0
    for i, rel in enumerate(y_true):
        if rel:
            hits += 1
            sum_precisions += hits / (i + 1)
    return sum_precisions / max(hits, 1)

def compute_reciprocal_rank(y_true):
    for i, rel in enumerate(y_true, start=1):  # ranks start at 1
        if rel:
            return 1.0 / i
    return 0.0  # no relevant item found in top K


### Embed error traces and match them against code chunk embeddings



Running each eligible bug through the model and embedding them, after then running cosine similarity to determine which files they think it should be changed

In [4]:
projects = get_projects()
all_ap = []
all_rr = []

for project in projects:
    bugs = get_bugs(project)
    # embedding_index_path = f"tmp/ast/embeddings/index_{project}.faiss"
    bug_result_path = os.path.abspath(f"tmp/ast/results/bug_results_{project}.json")
    filtered_bugs = []
    error_texts = []

    # First pass: filter bugs and collect error traces
    for bug in bugs:
            
        changed_files = parse_changed_files(project, bug)
        if len(changed_files) > 1:
            continue
        info = get_bug_info(project, bug)
        error = extract_python_tracebacks(project, bug)
        if error:
            filtered_bugs.append(bug)
            error_texts.append(error)

    # Batch encode
    if error_texts:
        error_embeddings = embed(error_texts, batch_size=BATCH_SIZE, show_progress_bar=True)

        output = []

        for bug, emb in zip(filtered_bugs, error_embeddings):
            code_chunks_path = f"dataset/{project}/{bug}/code_chunks.json"
            with open(code_chunks_path, "r") as f:
                code_chunks = json.loads(f.read())
            embedding_path = f"dataset/{project}/{bug}/embedding.npy"
            embeddings = np.load(embedding_path)
            index = index_embeddings(embeddings)
            D, I = index.search(np.array([emb]).astype("float32"), k=K)
            search_results = {"index": bug, "files": []}
            for idx in I[0]:
                result = code_chunks[idx]
                search_results["files"].append(
                    {"file": result["file"], "function": result["name"]}
                )
            output.append(search_results)
            
            changed_file = parse_changed_files(project, bug)[0].split("/")[-1]
            y_true = [1 if code_chunks[idx]["file"].split("/")[-1] == changed_file else 0 for idx in I[0]]
            ap = compute_average_precision(y_true)
            rr = compute_reciprocal_rank(y_true)

            all_ap.append(ap)
            all_rr.append(rr)

        with open(bug_result_path, "w", encoding="utf-8") as f:
            json.dump(output, f, indent=2)

Embedding:   0%|          | 0/23 [00:00<?, ?it/s]

Embedding: 100%|██████████| 34/34 [00:26<00:00,  1.30it/s]


### Generate results in a human readable way "results_K.json"

Using that predictions to calculate success rate and show where each search was successful or not.

In [5]:

results_folder = os.path.abspath("tmp/ast/results")
results_files = os.listdir(results_folder)
def analyze(k):
    results = {
        "K": k,
        "model_name": MODEL_NAME,
        "searches_failed": [],  # [{project_name, bug_id, detected_files: list, actual_file}]
        "searches_passed": [],  # [{project_name, bug_id, detected_files: list, actual_files}]
        "success_rate": 0,  # out of 100 (including bugs skipped)
        "MAP": np.mean(all_ap),
        "MRR": np.mean(all_rr),
        "success_projects": {},  # {project_name, success_rate, success_rate_no_skip}
    }
    success_projects_tmp = {}
    count = 0
    maximum_bugs = 0
    for project in results_files:
        project_name = project.replace("bug_results_", "").replace(".json", "")
        bugs = get_bugs(project_name)
        maximum_bugs += len(bugs)
        success_projects_tmp[project_name] = {
            "failed": 0,
            "passed": 0,
            "total": 0,
        }

        with open(f"{results_folder}/{project}", "r", encoding="utf-8") as result_file:
            data = json.loads(result_file.read())
            for search_result in data:
                changed_file = parse_changed_files(project_name, search_result["index"])[0].split("/")[-1]
                files_predicted = [
                    obj["file"].split("/")[-1] for obj in search_result["files"][:k]
                ]
                if changed_file in files_predicted:
                    results["searches_passed"].append(
                        {
                            "project_name": project_name,
                            "bug_id": search_result["index"],
                            "detected_files": files_predicted,
                            "actual_file": changed_file,
                        }
                    )
                    success_projects_tmp[project_name]["passed"] += 1
                    success_projects_tmp[project_name]["total"] += 1
                else:
                    results["searches_failed"].append(
                        {
                            "project_name": project_name,
                            "bug_id": search_result["index"],
                            "detected_files": files_predicted,
                            "actual_file": changed_file,
                        }
                    )
                    success_projects_tmp[project_name]["failed"] += 1
                    success_projects_tmp[project_name]["total"] += 1
                    


        searches_counted = (
            success_projects_tmp[project_name]["passed"]
            + success_projects_tmp[project_name]["failed"]
        )

        results["success_projects"][project_name] = {
            "success_rate": success_projects_tmp[project_name]["passed"] / searches_counted,
        } | success_projects_tmp[project_name]

    results["success_rate"] = len(results["searches_passed"]) / (len(results["searches_failed"]) + len(results["searches_passed"]))
    with open(os.path.abspath(f"results_{k}.json"), "w", encoding="utf-8") as file:
        file.write(json.dumps(results, indent=2))
        
analyze(1) # 1 = K value
analyze(5) # 5 = K value
analyze(10) # 10 = K value

### An easier way to show the results and is used to create tables in the report

In [9]:
Ks = [1, 5, 10]

def show(k: int):
    # Load your data
    with open(f"results_{k}.json", "r") as f:
        data = json.load(f)

    # Desired project order
    ordered_projects = [
        "youtube-dl",
        "keras",
        "matplotlib",
        "black",
        "thefuck",
        "scrapy",
        "pandas",
        "luigi",
        "ansible"
    ]

    # Print header
    print(f"{'Project':<15} {'Passed':>6} {'Failed':>6} {'Total':>6} {'Success Rate':>13}")
    print("=" * 50)

    # Print each project in order
    for project in ordered_projects:
        stats = data["success_projects"][project]
        passed = stats["passed"]
        failed = stats["failed"]
        total = stats["total"]
        rate = stats["success_rate"]
        print(f"{project:<15} {passed:>6} {failed:>6} {total:>6} {rate:>12.2f}%")

    # Print overall success rate
    print("\nOverall Success Rate:", f"{data['success_rate'] * 100:.2f}%")

for k in Ks:
    show(k)

Project         Passed Failed  Total  Success Rate
0.9411764705882353
0.6470588235294118
0.6521739130434783
0.7894736842105263
0.9
0.9459459459459459
0.6129032258064516
1.0

Overall Success Rate: 76.13%
